In [13]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score


In [4]:
df = pd.read_csv('data_week_3.csv')

df.columns = df.columns.str.lower().str.replace(' ', '_')
categorical_columns = list(df.dtypes[df.dtypes == 'object'].index)

for c in categorical_columns:
    df[c] = df[c].str.lower().str.replace(' ', '_')

df['totalcharges'] = pd.to_numeric(df['totalcharges'], errors='coerce')
df['seniorcitizen'] = df['seniorcitizen'].astype(object)
df['totalcharges'] = df['totalcharges'].fillna(0)
df['churn'] = (df['churn'] == 'yes').astype(int)

In [5]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=1)

In [7]:
numerical = list(df.select_dtypes(include=['int64', 'float64']).columns)
categorical = list(df.select_dtypes(include='object').columns)

In [8]:
def train(df_train, y_train, C=1.0):
    dicts = df_train[categorical+numerical].to_dict(orient='records')

    dv = DictVectorizer(sparse=False)
    X_train = dv.fit_transform(dicts)

    model = LogisticRegression(C=C, max_iter=1000)
    model.fit(X_train, y_train)

    return dv, model

In [9]:
def predict(df, dv, model):
    dicts = df[categorical+numerical].to_dict(orient='records')

    X = dv.transform(dicts)
    y_pred = model.predict_proba(X)[:, 1]
    
    return y_pred

In [10]:
C = 1.0
n_splits = 5

In [14]:
kfold = KFold(n_splits=n_splits, shuffle=True, random_state=1)

scores = []

for train_idx, val_idx in kfold.split(df_full_train):
    df_train = df_full_train.iloc[train_idx]
    df_val = df_full_train.iloc[val_idx]

    y_train = df_train['churn'].values
    y_val = df_val['churn'].values

    dv, model = train(df_train, y_train, C)
    y_pred = predict(df_val, dv, model)

    auc = roc_auc_score(y_val, y_pred)
    scores.append(auc)

print('C=%s %.3f +- %.3f' % (C, np.mean(scores), np.std(scores)))

C=1.0 0.838 +- 0.007


In [15]:
scores

[0.8444434850185648,
 0.845138475036419,
 0.831944611679711,
 0.8285057076165505,
 0.8403564368861977]

In [18]:
dv, model = train(df_full_train, df_full_train['churn'].values, C=1)
y_pred = predict(df_test, dv, model)
y_test = df_test['churn'].values

auc = roc_auc_score(y_test, y_pred)
auc

0.8591371726954619

In [19]:
import pickle

In [20]:
output_file = f'model_C={C}.bin'

In [21]:
with open(output_file, 'wb') as f_out:
    pickle.dump((dv, model), f_out)

In [1]:
import pickle

In [3]:
model_file = 'model_C=1.0.bin'

In [4]:
with open(model_file, 'rb') as f_in:
    (dv, model) = pickle.load(f_in)

In [5]:
dv, model

(DictVectorizer(sparse=False), LogisticRegression(C=1, max_iter=1000))

In [6]:
customer = {
    'gender': 'female',
    'seniorcitizen': 0,
    'partner': 'yes',
    'dependents': 'no',
    'phoneservice': 'no',
    'multiplelines': 'no_phone_service',
    'internetservice': 'dsl',
    'onlinesecurity': 'no',
    'onlinebackup': 'yes',
    'deviceprotection': 'no',
    'techsupport': 'no',
    'streamingtv': 'no',
    'streamingmovies': 'no',
    'contract': 'month-to-month',
    'paperlessbilling': 'yes',
    'paymentmethod': 'electronic_check',
    'tenure': 1,
    'monthlycharges': 29.85,
    'totalcharges': 29.85
}

In [9]:
X = dv.transform(customer)

array([[ 1.  ,  0.  ,  0.  , ...,  0.  ,  1.  , 29.85]])

In [12]:
y_pred = model.predict_proba(X)[0, 1]
y_pred

0.6578654534872822